## Utilisation de LINQ et Z3 pour la résolution de problèmes

### Installation de Z3.Linq

On appelle le package Nuget correspondant. 




In [7]:
#r "nuget: Z3.Linq"

Installed Packages Z3.Linq, 2.0.1

### Exemple court

In [8]:
using Z3.Linq; 

using (var ctx = new Z3Context())
      {
        var theorem = from t in ctx.NewTheorem<Symbols<int, int, int, int, int>>()
              where t.X1 - t.X2 >= 1
              where t.X1 - t.X2 <= 3
              where t.X1 == (2 * t.X3) + t.X5
              where t.X3 == t.X5
              where t.X2 == 6 * t.X4
              select t;
var solution = theorem.Solve();
Console.WriteLine("X1 = {0}, X2 = {1}, X3 = {2}, X4 = {3}, X5 = {4}", solution.X1, solution.X2, solution.X3, solution.X4, solution.X5);

}

X1 = 3, X2 = 0, X3 = 1, X4 = 0, X5 = 1


### Classe de missionnaires et cannibales

###	Affirmer les contraintes

L’affirmation des contraintes des arbres d’expression LINQ sur la classe d’état et l’étape suivante. On se base pour ça sur la classe Theorem de LINQ To Z3 :


In [15]:
public class CanibalsAndMissionaries: CanibalsAndMissionariesBase
{
    
    // Le nombre de canibales et missionaires (3 dans le problème original)
    public int NbMissionaries { get; set; } = 3;
    // La taille de la barque (2 dans le projet original)
    public int SizeBoat { get; set; } = 2;

    // La longueur du chemin calculé
    private int _length;

    //La propriété qui permet d'accéder à la taille du chemin dans Z3
    public int Length
    {
      get => _length;
      set
      {
        _length = value;
        // Quand la longueur est déterminée par Z3, on initialise les tableaux pour pouvoir récupérer les valeurs
        Canibals = new int[value];
        Missionaries = new int[value];
      }
    }

    // Un tableau contenant à chaque étape le nombre de canibales sur la berge de départ
    public int[] Canibals { get; set; }
    // Un tableau contenant à chaque étape le nombre de missionaires sur la berge de départ
    public int[] Missionaries { get; set; }

    /// <summary>
    /// Une représentation lisible de la solution proposée
    /// </summary>
    /// <returns>une chaine de caractère ou chaque ligne est une étape du chemin</returns>
    public override string ToString()
    {
      var sb = new StringBuilder();
      for (int i = 0; i < Canibals.Length; i++)
      {
        sb.AppendLine($"{i + 1} - (({Missionaries[i]}M, {Canibals[i]}C, {1 - i % 2}), ({(i % 2)}, {NbMissionaries - Missionaries[i]}M, {NbMissionaries - Canibals[i]}C))");
      }

      return sb.ToString();

    }
    
    /// <summary>
    /// La méthode qui permet la création du théorème associé au problème
    /// </summary>
    /// <param name="context">Le contexte Z3 qui devra interpréter les contraintes</param>
    /// <param name="entity">Une valeur du problème servant de modèle pour définir les paramètres principaux</param>
    /// <returns>Un théorème de notre environnement qui peut être filtré et résolu</returns>
    public static Theorem<CanibalsAndMissionaries> Create(Z3Context context, CanibalsAndMissionaries entity)
    {
      // On créée une instance du théorème, sans contraintes, puis on va rajouter les contraintes une à une
      var theorem = context.NewTheorem<CanibalsAndMissionaries>();
      
    // Contraintes globales
      // On récupère les contraintes globales qui seront injectées sous forme de constante dans la lambda expression
      var sizeBoat = entity.SizeBoat;
      int nbMissionaries = entity.NbMissionaries;
      int maxlength = entity.Length;
      // Etat initial
      theorem = theorem.Where(caM => caM.NbMissionaries == nbMissionaries);
      theorem = theorem.Where(caM => caM.SizeBoat == sizeBoat);
        theorem = theorem.Where(caM => caM.Missionaries[0] == caM.NbMissionaries && caM.Canibals[0] == caM.NbMissionaries);

      //Modèle de transition
      // On filtre à chaque étape selon les actions possible
      for (int iclosure = 0; iclosure < maxlength; iclosure++)
      {
        var i = iclosure;
        //Les deux rives contiennent entre 0 et 3 personnes
        theorem = theorem.Where(caM => caM.Canibals[i] >= 0
                                       && caM.Canibals[i] <= caM.NbMissionaries
                                       && caM.Missionaries[i] >= 0
                                       && caM.Missionaries[i] <= caM.NbMissionaries);
        if (i % 2 == 0)
        {
          // Aux itérations paires, la rive de départ perd entre 1 et SizeBoat personnes 
          theorem = theorem.Where(caM => caM.Canibals[i + 1] <= caM.Canibals[i]
                                         && caM.Missionaries[i + 1] <= caM.Missionaries[i]
                                         && caM.Canibals[i + 1] + caM.Missionaries[i + 1] - caM.Canibals[i] - caM.Missionaries[i] < 0
                                         && caM.Canibals[i + 1] + caM.Missionaries[i + 1] - caM.Canibals[i] - caM.Missionaries[i] >= -caM.SizeBoat);
        }
        else
        {
          // Aux itérations impaires, la rive de départ gagne entre 1 et SizeBoat personnes 
          theorem = theorem.Where(caM =>
                                    caM.Canibals[i + 1] >= caM.Canibals[i]
                                    && caM.Missionaries[i + 1] >= caM.Missionaries[i]
                                    && caM.Canibals[i + 1] + caM.Missionaries[i + 1] - caM.Canibals[i] - caM.Missionaries[i] > 0
                                    && caM.Canibals[i + 1] + caM.Missionaries[i + 1] - caM.Canibals[i] - caM.Missionaries[i] <= caM.SizeBoat);

        }

        //Jamais moins de missionnaire que de cannibal sur chacune des rives
        theorem = theorem.Where(caM => (caM.Missionaries[i] == 0 || (caM.Missionaries[i] >= caM.Canibals[i]))
                                 && (caM.Missionaries[i] == caM.NbMissionaries || ((caM.NbMissionaries - caM.Missionaries[i]) >= (caM.NbMissionaries - caM.Canibals[i]))));

      }


        // Test de but
      // A l'arrivée, plus personne sur la rive de départ
      theorem = theorem.Where(
        caM => caM.Length > 0
               && caM.Length < maxlength
               && caM.Missionaries[caM.Length - 1] == 0
               && caM.Canibals[caM.Length - 1] == 0
      );


      return theorem;
    }
    
}

###	Obtenir la solution 
LINQ To Z3 nous donne la solution sous forme d’ un objet POCO (Plain Old CLR Object) du type de paramètre générique T du théorème. 


In [16]:
using System.Diagnostics;
var stopWatch = new Stopwatch();
      stopWatch.Start();
      TimeSpan debutChrono;
    // Solving Canibals & Missionaires
      var can = new CanibalsAndMissionaries(){NbMissionaries = 3, SizeBoat = 2, Length = 30};

      using (var ctx = new Z3Context())
      {
        var theorem = CanibalsAndMissionaries.Create(ctx, can);

        debutChrono = stopWatch.Elapsed;

        //Print(theorem);
        var result = theorem.Solve();

        // affichage du résultat
        display($"Durée Cannibales et Missionaires {stopWatch.Elapsed - debutChrono}");
        display(result);

      }


Durée Cannibales et Missionaires 00:00:00.2848501

NbMissionaries,3
SizeBoat,2
Length,22
Canibals,"[ 3, 2, 3, 2, 3, 2, 2, 2, 2, 0, 1, 0, 2, 0, 1, 1, 2, 2, 3, 1 ... (2 more) ]"
Missionaries,"[ 3, 2, 3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 1, 2, 0, 0, 0 ... (2 more) ]"


### Recherche de la solution la plus courte

In [17]:
using System.Diagnostics;
var stopWatch = new Stopwatch();
      stopWatch.Start();
      TimeSpan debutChrono;
    // Solving Canibals & Missionaires
      var can = new CanibalsAndMissionaries(){NbMissionaries = 3, SizeBoat = 2, Length = 30};

      using (var ctx = new Z3Context())
      {
        var theorem = CanibalsAndMissionaries.Create(ctx, can);

        debutChrono = stopWatch.Elapsed;

        //Print(theorem);
        var result = theorem.Optimize(Optimization.Minimize, objMnC => objMnC.Length);

        // affichage du résultat
        display($"Durée Cannibales et Missionaires {stopWatch.Elapsed - debutChrono}");
        display(result);

      }




Durée Cannibales et Missionaires 00:00:00.1130628

NbMissionaries,3
SizeBoat,2
Length,12
Canibals,"[ 3, 1, 2, 0, 1, 1, 2, 2, 3, 1, 1, 0 ]"
Missionaries,"[ 3, 3, 3, 3, 3, 1, 2, 0, 0, 0, 1, 0 ]"


In [20]:
using System.Diagnostics;
var stopWatch = new Stopwatch();
      stopWatch.Start();
      TimeSpan debutChrono;
    // Solving Canibals & Missionaires
      var can = new CanibalsAndMissionaries(){NbMissionaries = 30, SizeBoat = 7, Length = 100};

      using (var ctx = new Z3Context())
      {
        var theorem = CanibalsAndMissionaries.Create(ctx, can);

        debutChrono = stopWatch.Elapsed;

        //Print(theorem);
        var result = theorem.Optimize(Optimization.Minimize, objMnC => objMnC.Length);

        // affichage du résultat
        display($"Durée Cannibales et Missionaires {stopWatch.Elapsed - debutChrono}");
        display(result);

      }




Error: Command cancelled.